This originally was the MMAction2 Tutorial. I gutted it for our purposes

In [ ]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
# install dependencies: (use cu101 because colab has CUDA 10.1)
!pip install -U torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

# install mmcv-full thus we could use CUDA operators
!pip install mmcv-full==latest+torch1.5.0+cu101 -f https://download.openmmlab.com/mmcv/dist/index.html

# Install mmaction2
!rm -rf mmaction2
!git clone https://github.com/open-mmlab/mmaction2.git
%cd mmaction2

!pip install -e .

# Install some optional requirements
!pip install -r requirements/optional.txt

Looking in links: https://download.pytorch.org/whl/torch_stable.html
Requirement already up-to-date: torch==1.5.1+cu101 in /usr/local/lib/python3.6/dist-packages (1.5.1+cu101)
Requirement already up-to-date: torchvision==0.6.1+cu101 in /usr/local/lib/python3.6/dist-packages (0.6.1+cu101)
Looking in links: https://download.openmmlab.com/mmcv/dist/index.html
  Using cached https://download.openmmlab.com/mmcv/dist/latest/torch1.5.0/cu101/mmcv_full-latest%2Btorch1.5.0%2Bcu101-cp36-cp36m-manylinux1_x86_64.whl
  Found existing installation: mmcv-full 1.2.0
    Uninstalling mmcv-full-1.2.0:
      Successfully uninstalled mmcv-full-1.2.0
Cloning into 'mmaction2'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 7596 (delta 0), reused 1 (delta 0), pack-reused 7588
Receiving objects: 100% (7596/7596), 33.53 MiB | 37.44 MiB/s, done.
Resolving deltas: 100% (5329/5329), done.
/content/mmaction2
Obtaining

In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMAction2 installation
import mmaction
print(mmaction.__version__)

# Check MMCV installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.5.1+cu101 True
0.8.0
10.1
GCC 7.3


# Loading Data
This is where the data is loaded, we will need to overwrite this part

In [ ]:
!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x3_100e_kinetics400_rgb/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth \
      -O checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth

--2020-11-20 09:38:27--  https://download.openmmlab.com/mmaction/recognition/tsn/tsn_r50_1x1x3_100e_kinetics400_rgb/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.252.96.35
Connecting to download.openmmlab.com (download.openmmlab.com)|47.252.96.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 97579339 (93M) [application/octet-stream]
Saving to: ‘checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth’

checkpoints/tsn_r50 100%[===================>]  93.06M  8.48MB/s    in 11s     

2020-11-20 09:38:40 (8.49 MB/s) - ‘checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth’ saved [97579339/97579339]



In [ ]:
from mmaction.apis import inference_recognizer, init_recognizer

# Choose to use a config and initialize the recognizer
config = 'configs/recognition/tsn/tsn_r50_video_inference_1x1x3_100e_kinetics400_rgb.py'
# Setup a checkpoint file to load
checkpoint = 'checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth'
# Initialize the recognizer
model = init_recognizer(config, checkpoint, device='cuda:0')

In [ ]:
# Use the recognizer to do inference
video = 'demo/demo.mp4'
label = 'demo/label_map_k400.txt'
results = inference_recognizer(model, video, label)

In [ ]:
# Let's show the results
for result in results:
    print(f'{result[0]}: ', result[1])

arm wrestling:  29.616438
rock scissors paper:  10.754841
shaking hands:  9.908401
clapping:  9.189913
massaging feet:  8.305306


## Train a recognizer on customized dataset

To train a new recognizer, there are usually three things to do:
1. Support a new dataset
2. Modify the config
3. Train a new recognizer

### Support a new dataset

In this tutorial, we gives an example to convert the data into the format of existing datasets. Other methods and more advanced usages can be found in the [doc](/docs/tutorials/new_dataset.md)

Firstly, let's download a tiny dataset obtained from [Kinetics-400](https://deepmind.com/research/open-source/open-source-datasets/kinetics/). We select 30 videos with their labels as train dataset and 10 videos with their labels as test dataset.

In [ ]:
# download, decompress the data
!rm kinetics400_tiny.zip*
!rm -rf kinetics400_tiny
!wget https://download.openmmlab.com/mmaction/kinetics400_tiny.zip
!unzip kinetics400_tiny.zip > /dev/null

rm: cannot remove 'kinetics400_tiny.zip*': No such file or directory
--2020-11-20 09:38:48--  https://download.openmmlab.com/mmaction/kinetics400_tiny.zip
Resolving download.openmmlab.com (download.openmmlab.com)... 47.252.96.35
Connecting to download.openmmlab.com (download.openmmlab.com)|47.252.96.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18308682 (17M) [application/zip]
Saving to: ‘kinetics400_tiny.zip’

kinetics400_tiny.zi 100%[===================>]  17.46M  8.59MB/s    in 2.0s    

2020-11-20 09:38:51 (8.59 MB/s) - ‘kinetics400_tiny.zip’ saved [18308682/18308682]



In [ ]:
# Check the directory structure of the tiny data

# Install tree first
!apt-get -q install tree
!tree kinetics400_tiny

Reading package lists...
Building dependency tree...
Reading state information...
tree is already the newest version (1.7.0-5).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.
kinetics400_tiny
├── kinetics_tiny_train_video.txt
├── kinetics_tiny_val_video.txt
├── train
│   ├── 27_CSXByd3s.mp4
│   ├── 34XczvTaRiI.mp4
│   ├── A-wiliK50Zw.mp4
│   ├── D32_1gwq35E.mp4
│   ├── D92m0HsHjcQ.mp4
│   ├── DbX8mPslRXg.mp4
│   ├── FMlSTTpN3VY.mp4
│   ├── h10B9SVE-nk.mp4
│   ├── h2YqqUhnR34.mp4
│   ├── iRuyZSKhHRg.mp4
│   ├── IyfILH9lBRo.mp4
│   ├── kFC3KY2bOP8.mp4
│   ├── LvcFDgCAXQs.mp4
│   ├── O46YA8tI530.mp4
│   ├── oMrZaozOvdQ.mp4
│   ├── oXy-e_P_cAI.mp4
│   ├── P5M-hAts7MQ.mp4
│   ├── phDqGd0NKoo.mp4
│   ├── PnOe3GZRVX8.mp4
│   ├── R8HXQkdgKWA.mp4
│   ├── RqnKtCEoEcA.mp4
│   ├── soEcZZsBmDs.mp4
│   ├── TkkZPZHbAKA.mp4
│   ├── T_TMNGzVrDk.mp4
│   ├── WaS0qwP46Us.mp4
│   ├── Wh_YPQdH1Zg.mp4
│   ├── WWP5HZJsg-o.mp4
│   ├── xGY2dP0YUjA.mp4
│   ├── yLC9CtWU5ws.mp4
│   └── ZQV4U2KQ370

In [ ]:
# After downloading the data, we need to check the annotation format
!cat kinetics400_tiny/kinetics_tiny_train_video.txt

D32_1gwq35E.mp4 0
iRuyZSKhHRg.mp4 1
oXy-e_P_cAI.mp4 0
34XczvTaRiI.mp4 1
h2YqqUhnR34.mp4 0
O46YA8tI530.mp4 0
kFC3KY2bOP8.mp4 1
WWP5HZJsg-o.mp4 1
phDqGd0NKoo.mp4 1
yLC9CtWU5ws.mp4 0
27_CSXByd3s.mp4 1
IyfILH9lBRo.mp4 1
T_TMNGzVrDk.mp4 1
TkkZPZHbAKA.mp4 0
PnOe3GZRVX8.mp4 1
soEcZZsBmDs.mp4 1
FMlSTTpN3VY.mp4 1
WaS0qwP46Us.mp4 0
A-wiliK50Zw.mp4 1
oMrZaozOvdQ.mp4 1
ZQV4U2KQ370.mp4 0
DbX8mPslRXg.mp4 1
h10B9SVE-nk.mp4 1
P5M-hAts7MQ.mp4 0
R8HXQkdgKWA.mp4 0
D92m0HsHjcQ.mp4 0
RqnKtCEoEcA.mp4 0
LvcFDgCAXQs.mp4 0
xGY2dP0YUjA.mp4 0
Wh_YPQdH1Zg.mp4 0


According to the format defined in [`VideoDataset`](./datasets/video_dataset.py), each line indicates a sample video with the filepath and label, which are split with a whitespace.

### Modify the config

In the next step, we need to modify the config for the training.
To accelerate the process, we finetune a recognizer using a pre-trained recognizer.

In [ ]:
from mmcv import Config
cfg = Config.fromfile('./configs/recognition/tsn/tsn_r50_video_1x1x8_100e_kinetics400_rgb.py')

Given a config that trains a TSN model on kinetics400-full dataset, we need to modify some values to use it for training TSN on Kinetics400-tiny dataset.


In [ ]:
from mmcv.runner import set_random_seed

# Modify dataset type and path
cfg.dataset_type = 'VideoDataset'
cfg.data_root = 'kinetics400_tiny/train/'
cfg.data_root_val = 'kinetics400_tiny/val/'
cfg.ann_file_train = 'kinetics400_tiny/kinetics_tiny_train_video.txt'
cfg.ann_file_val = 'kinetics400_tiny/kinetics_tiny_val_video.txt'
cfg.ann_file_test = 'kinetics400_tiny/kinetics_tiny_val_video.txt'

cfg.data.test.type = 'VideoDataset'
cfg.data.test.ann_file = 'kinetics400_tiny/kinetics_tiny_val_video.txt'
cfg.data.test.data_prefix = 'kinetics400_tiny/val/'

cfg.data.train.type = 'VideoDataset'
cfg.data.train.ann_file = 'kinetics400_tiny/kinetics_tiny_train_video.txt'
cfg.data.train.data_prefix = 'kinetics400_tiny/train/'

cfg.data.val.type = 'VideoDataset'
cfg.data.val.ann_file = 'kinetics400_tiny/kinetics_tiny_val_video.txt'
cfg.data.val.data_prefix = 'kinetics400_tiny/val/'

# The flag is used to determine whether it is omnisource training
cfg.setdefault('omnisource', False)
# Modify num classes of the model in cls_head
cfg.model.cls_head.num_classes = 2
# We can use the pre-trained TSN model
cfg.load_from = './checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth'

# Set up working dir to save files and logs.
cfg.work_dir = './tutorial_exps'

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.data.videos_per_gpu = cfg.data.videos_per_gpu // 16
cfg.optimizer.lr = cfg.optimizer.lr / 8 / 16
cfg.total_epochs = 30

# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 10
# We can set the log print interval to reduce the the times of printing log
cfg.log_config.interval = 5

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)


# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')


Config:
model = dict(
    type='Recognizer2D',
    backbone=dict(
        type='ResNet',
        pretrained='torchvision://resnet50',
        depth=50,
        norm_eval=False),
    cls_head=dict(
        type='TSNHead',
        num_classes=2,
        in_channels=2048,
        spatial_type='avg',
        consensus=dict(type='AvgConsensus', dim=1),
        dropout_ratio=0.4,
        init_std=0.01))
train_cfg = None
test_cfg = dict(average_clips=None)
dataset_type = 'VideoDataset'
data_root = 'kinetics400_tiny/train/'
data_root_val = 'kinetics400_tiny/val/'
ann_file_train = 'kinetics400_tiny/kinetics_tiny_train_video.txt'
ann_file_val = 'kinetics400_tiny/kinetics_tiny_val_video.txt'
ann_file_test = 'kinetics400_tiny/kinetics_tiny_val_video.txt'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_bgr=False)
train_pipeline = [
    dict(type='DecordInit'),
    dict(type='SampleFrames', clip_len=1, frame_interval=1, num_clips=8),
    dict(type='DecordDeco

### Train a new recognizer

Finally, lets initialize the dataset and recognizer, then train a new recognizer!

In [ ]:
import os.path as osp

from mmaction.datasets import build_dataset
from mmaction.models import build_model
from mmaction.apis import train_model

import mmcv

# Build the dataset
datasets = [build_dataset(cfg.data.train)]

# Build the recognizer
model = build_model(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_model(model, datasets, cfg, distributed=False, validate=True)

2020-11-20 09:38:54,909 - mmaction - INFO - These parameters in pretrained checkpoint are not loaded: {'fc.weight', 'fc.bias'}
2020-11-20 09:38:54,960 - mmaction - INFO - load checkpoint from ./checkpoints/tsn_r50_1x1x3_100e_kinetics400_rgb_20200614-e508be42.pth
2020-11-20 09:38:55,052 - mmaction - WARNING - The model and loaded state dict do not match exactly

size mismatch for cls_head.fc_cls.weight: copying a param with shape torch.Size([400, 2048]) from checkpoint, the shape in current model is torch.Size([2, 2048]).
size mismatch for cls_head.fc_cls.bias: copying a param with shape torch.Size([400]) from checkpoint, the shape in current model is torch.Size([2]).
2020-11-20 09:38:55,056 - mmaction - INFO - Start running, host: root@74e02ee9f123, work_dir: /content/mmaction2/tutorial_exps
2020-11-20 09:38:55,061 - mmaction - INFO - workflow: [('train', 1)], max: 30 epochs
2020-11-20 09:38:59,709 - mmaction - INFO - Epoch [1][5/15]	lr: 7.813e-05, eta: 0:06:52, time: 0.927, data_time:

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 10/10, 4.0 task/s, elapsed: 3s, ETA:     0s

2020-11-20 09:39:33,791 - mmaction - INFO - Evaluating top_k_accuracy ...
2020-11-20 09:39:33,793 - mmaction - INFO - 
top1_acc	0.7000
top5_acc	1.0000
2020-11-20 09:39:33,794 - mmaction - INFO - Evaluating mean_class_accuracy ...
2020-11-20 09:39:33,797 - mmaction - INFO - 
mean_acc	0.7000
2020-11-20 09:39:33,798 - mmaction - INFO - Now best checkpoint is epoch_5.pth
2020-11-20 09:39:33,801 - mmaction - INFO - Epoch(val) [5][15]	top1_acc: 0.7000, top5_acc: 1.0000, mean_class_accuracy: 0.7000
2020-11-20 09:39:39,131 - mmaction - INFO - Epoch [6][5/15]	lr: 7.813e-05, eta: 0:03:09, time: 1.065, data_time: 0.832, memory: 2918, top1_acc: 0.5000, top5_acc: 1.0000, loss_cls: 0.6696, loss: 0.6696, grad_norm: 11.0206
2020-11-20 09:39:40,263 - mmaction - INFO - Epoch [6][10/15]	lr: 7.813e-05, eta: 0:03:00, time: 0.226, data_time: 0.002, memory: 2918, top1_acc: 0.6000, top5_acc: 1.0000, loss_cls: 0.6824, loss: 0.6824, grad_norm: 11.8897
2020-11-20 09:39:41,147 - mmaction - INFO - Epoch [6][15/15]

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 10/10, 4.2 task/s, elapsed: 2s, ETA:     0s

2020-11-20 09:40:12,776 - mmaction - INFO - Evaluating top_k_accuracy ...
2020-11-20 09:40:12,778 - mmaction - INFO - 
top1_acc	0.9000
top5_acc	1.0000
2020-11-20 09:40:12,778 - mmaction - INFO - Evaluating mean_class_accuracy ...
2020-11-20 09:40:12,782 - mmaction - INFO - 
mean_acc	0.9000
2020-11-20 09:40:12,783 - mmaction - INFO - Now best checkpoint is epoch_10.pth
2020-11-20 09:40:12,786 - mmaction - INFO - Epoch(val) [10][15]	top1_acc: 0.9000, top5_acc: 1.0000, mean_class_accuracy: 0.9000
2020-11-20 09:40:17,313 - mmaction - INFO - Epoch [11][5/15]	lr: 7.813e-05, eta: 0:02:24, time: 0.904, data_time: 0.671, memory: 2918, top1_acc: 0.8000, top5_acc: 1.0000, loss_cls: 0.5357, loss: 0.5357, grad_norm: 8.7986
2020-11-20 09:40:18,877 - mmaction - INFO - Epoch [11][10/15]	lr: 7.813e-05, eta: 0:02:20, time: 0.313, data_time: 0.097, memory: 2918, top1_acc: 0.8000, top5_acc: 1.0000, loss_cls: 0.5829, loss: 0.5829, grad_norm: 10.5930
2020-11-20 09:40:19,797 - mmaction - INFO - Epoch [11][15

...

2020-11-20 09:41:29,078 - mmaction - INFO - Evaluating top_k_accuracy ...
2020-11-20 09:41:29,079 - mmaction - INFO - 
top1_acc	0.9000
top5_acc	1.0000
2020-11-20 09:41:29,080 - mmaction - INFO - Evaluating mean_class_accuracy ...
2020-11-20 09:41:29,084 - mmaction - INFO - 
mean_acc	0.9000
2020-11-20 09:41:29,086 - mmaction - INFO - Epoch(val) [20][15]	top1_acc: 0.9000, top5_acc: 1.0000, mean_class_accuracy: 0.9000
2020-11-20 09:41:33,520 - mmaction - INFO - Epoch [21][5/15]	lr: 7.813e-05, eta: 0:01:09, time: 0.886, data_time: 0.653, memory: 2918, top1_acc: 0.8000, top5_acc: 1.0000, loss_cls: 0.4515, loss: 0.4515, grad_norm: 8.3883
2020-11-20 09:41:35,097 - mmaction - INFO - Epoch [21][10/15]	lr: 7.813e-05, eta: 0:01:06, time: 0.316, data_time: 0.095, memory: 2918, top1_acc: 0.8000, top5_acc: 1.0000, loss_cls: 0.4543, loss: 0.4543, grad_norm: 9.1107
2020-11-20 09:41:36,032 - mmaction - INFO - Epoch [21][15/15]	lr: 7.813e-05, eta: 0:01:03, time: 0.187, data_time: 0.001, memory: 2918, to

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 10/10, 4.4 task/s, elapsed: 2s, ETA:     0s

2020-11-20 09:42:06,788 - mmaction - INFO - Evaluating top_k_accuracy ...
2020-11-20 09:42:06,790 - mmaction - INFO - 
top1_acc	1.0000
top5_acc	1.0000
2020-11-20 09:42:06,791 - mmaction - INFO - Evaluating mean_class_accuracy ...
2020-11-20 09:42:06,796 - mmaction - INFO - 
mean_acc	1.0000
2020-11-20 09:42:06,797 - mmaction - INFO - Now best checkpoint is epoch_25.pth
2020-11-20 09:42:06,799 - mmaction - INFO - Epoch(val) [25][15]	top1_acc: 1.0000, top5_acc: 1.0000, mean_class_accuracy: 1.0000
2020-11-20 09:42:11,579 - mmaction - INFO - Epoch [26][5/15]	lr: 7.813e-05, eta: 0:00:33, time: 0.954, data_time: 0.725, memory: 2918, top1_acc: 0.9000, top5_acc: 1.0000, loss_cls: 0.3759, loss: 0.3759, grad_norm: 7.9644
2020-11-20 09:42:13,112 - mmaction - INFO - Epoch [26][10/15]	lr: 7.813e-05, eta: 0:00:30, time: 0.307, data_time: 0.114, memory: 2918, top1_acc: 0.5000, top5_acc: 1.0000, loss_cls: 0.5393, loss: 0.5393, grad_norm: 10.7580
2020-11-20 09:42:14,007 - mmaction - INFO - Epoch [26][15

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 10/10, 4.2 task/s, elapsed: 2s, ETA:     0s

2020-11-20 09:42:45,097 - mmaction - INFO - Evaluating top_k_accuracy ...
2020-11-20 09:42:45,098 - mmaction - INFO - 
top1_acc	1.0000
top5_acc	1.0000
2020-11-20 09:42:45,099 - mmaction - INFO - Evaluating mean_class_accuracy ...
2020-11-20 09:42:45,103 - mmaction - INFO - 
mean_acc	1.0000
2020-11-20 09:42:45,104 - mmaction - INFO - Epoch(val) [30][15]	top1_acc: 1.0000, top5_acc: 1.0000, mean_class_accuracy: 1.0000


### Understand the log
From the log, we can have a basic understanding the training process and know how well the recognizer is trained.

Firstly, the ResNet-50 backbone pre-trained on ImageNet is loaded, this is a common practice since training from scratch is more cost. The log shows that all the weights of the ResNet-50 backbone are loaded except the `fc.bias` and `fc.weight`.

Second, since the dataset we are using is small, we loaded a TSN model and finetune it for action recognition.
The original TSN is trained on original Kinetics-400 dataset which contains 400 classes but Kinetics-400 Tiny dataset only have 2 classes. Therefore, the last FC layer of the pre-trained TSN for classification has different weight shape and is not used.

Third, after training, the recognizer is evaluated by the default evaluation. The results show that the recognizer achieves 100% top1 accuracy and 100% top5 accuracy on the val dataset,
 
Not bad!

## Test the trained recognizer

After finetuning the recognizer, let's check the prediction results!

In [ ]:
from mmaction.apis import single_gpu_test
from mmaction.datasets import build_dataloader
from mmcv.parallel import MMDataParallel

# Build a test dataloader
dataset = build_dataset(cfg.data.test, dict(test_mode=True))
data_loader = build_dataloader(
        dataset,
        videos_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)
model = MMDataParallel(model, device_ids=[0])
outputs = single_gpu_test(model, data_loader)

eval_config = cfg.evaluation
eval_config.pop('interval')
eval_res = dataset.evaluate(outputs, **eval_config)
for name, val in eval_res.items():
    print(f'{name}: {val:.04f}')

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 10/10, 1.9 task/s, elapsed: 5s, ETA:     0s
Evaluating top_k_accuracy ...

top1_acc	1.0000
top5_acc	1.0000

Evaluating mean_class_accuracy ...

mean_acc	1.0000
top1_acc: 1.0000
top5_acc: 1.0000
mean_class_accuracy: 1.0000
